# [msticpy] ParentCommandLine powershell - base64 decode & ioc extraction

## Preparation for install msticpy

Example of phdrieger/mltk-container-golden-image-cpu:5.1.0 container:

$ mkdir /srv/app/model/customlib

$ pip install msticpy -t /srv/app/model/customlib

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [23]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import sys,os.path
sys.path.insert(0,"/srv/app/model/customlib")
import msticpy as mp

# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

# define the algorism name and model name
algo_name = "msticpy_powershell_ioc"
model_name = "process_b64_iocs_enrich"

## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [25]:
# NOT EXPORTED
from dsdlsupport import SplunkSearch as SplunkSearch
search = SplunkSearch.SplunkSearch()
df = search.as_df()

### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

index=botsv2 "powershell" "-enc" source="WinEventLog*Microsoft-Windows-Sysmon*Operational*"<br>
| where LIKE(ParentCommandLine,"%powershell%-enc%") | stats count by ParentCommandLine<br>
| fit MLTKContainer mode=stage algo=msticpy_powershell_ioc ParentCommandLine into app:process_b64_iocs_enrich<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [2]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [3]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage(model_name)

In [30]:
param

{'options': {'params': {'mode': 'stage', 'algo': 'msticpy_powershell_ioc'},
  'args': ['ParentCommandLine'],
  'feature_variables': ['ParentCommandLine'],
  'model_name': 'process_b64_iocs_enrich',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['ParentCommandLine']}

## Stage 2 - create and initialize a model

In [7]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 0
    model['desc'] = "msticpy ioc enrichment from process command line"
    return model

In [8]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'hyperparameter': 0, 'desc': 'msticpy ioc enrichment from process command line'}


## fit the model

In [9]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "passthru model creating the model's json file"}
    return info

In [10]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': "passthru model creating the model's json file"}


## apply the model

In [13]:
# NOT EXPORTED (just for debugging)
def join_result(dec_df,ioc_input):
    for input_string, iocs in ioc_input.items():
        j=0
        while j < len(dec_df['decoded_string']):
            if dec_df['decode_validated'][j] == input_string:
                dec_df.at[dec_df.index[j], 'ioc_ipv4'] = ",".join(iocs["ipv4"])
                dec_df.at[dec_df.index[j], 'ioc_url'] = ",".join(iocs["url"])
            j+=1    
    return dec_df[["ParentCommandLine","decode_validated","ioc_ipv4","ioc_url"]]

In [13]:
# NOT EXPORTED (just for debugging)
def format_ioc(ioc_df):
    ioc_input = {}
    for row in ioc_df.itertuples():
        if not row.Input in ioc_input:
            ioc_input[row.Input] = { "ipv4":[],"url":[] }
        if row.IoCType=="ipv4":
            ioc_input[row.Input]["ipv4"].append(row.Observable)
        elif row.IoCType=="url":
            url = row.Observable
        elif row.IoCType=="linux_path" and row.Observable[0]=="/" and ".php"in row.Observable.split(";")[0]:
            url_path = row.Observable.split(";")[0].rstrip("'").rstrip('"').rstrip()
            uri = url + url_path
            ioc_input[row.Input]["url"].append(uri)
    return ioc_input

In [16]:
# NOT EXPORTED
def extract_ioc(dec_df):
    i=0
    while i < len(dec_df["decoded_string"]):
        dec_df.at[dec_df.index[i], 'decode_validated'] = dec_df["decoded_string"][i].replace("\x00","")
        i+=1
    ioc_df = dec_df.mp.ioc_extract(columns=['decode_validated'],include_paths=True,\
                ioc_types=['ipv4','ipv6','dns','url','email','windows_path','linux_path','md5_hash','sha1_hash','sha256_hash'])
    return ioc_df

In [21]:
# apply your model
# returns the calculated results
def apply(model,df,param):  
    mp.init_notebook()
    
    # decode base64
    dec_df = mp.transform.base64unpack.unpack_df(data=df, column='ParentCommandLine')
    # remove nullbyte \x00
    i=0
    while i < len(dec_df["decoded_string"]):
        dec_df.at[dec_df.index[i], 'decode_validated'] = dec_df["decoded_string"][i].replace("\x00","")
        i+=1
    # extract ioc
    ioc_df = dec_df.mp.ioc_extract(columns=['decode_validated'],include_paths=True,\
                ioc_types=['ipv4','ipv6','dns','url','email','windows_path','linux_path','md5_hash','sha1_hash','sha256_hash'])
    # format ioc to dict
    ioc_input = {}
    for row in ioc_df.itertuples():
        if not row.Input in ioc_input:
            ioc_input[row.Input] = { "ipv4":[],"url":[] }
        if row.IoCType=="ipv4":
            ioc_input[row.Input]["ipv4"].append(row.Observable)
        elif row.IoCType=="url":
            url = row.Observable
        elif row.IoCType=="linux_path" and row.Observable[0]=="/" and ".php"in row.Observable.split(";")[0]:
            url_path = row.Observable.split(";")[0].rstrip("'").rstrip('"').rstrip()
            uri = url + url_path
            ioc_input[row.Input]["url"].append(uri)    
    
    # join result
    for input_string, iocs in ioc_input.items():
        j=0
        while j < len(dec_df['decoded_string']):
            if dec_df['decode_validated'][j] == input_string:
                dec_df.at[dec_df.index[j], 'ioc_ipv4'] = ",".join(iocs["ipv4"])
                dec_df.at[dec_df.index[j], 'ioc_url'] = ",".join(iocs["url"])
            j+=1 
    dec_ioc_df = dec_df[["ParentCommandLine","decode_validated","ioc_ipv4","ioc_url"]]

    # ipwhois enrichment 
    dec_ioc_enrich_df = dec_ioc_df.mp.whois(ip_column="ioc_ipv4")        

    return dec_ioc_enrich_df[["ParentCommandLine","decode_validated","ioc_ipv4","ioc_url","asn","asn_country_code","asn_description"]] #dec_ioc_enrich_df_completed   
    

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [13]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

In [14]:
save(model,algo_name+"_"+model_name)

{'hyperparameter': 0,
 'desc': 'msticpy ioc enrichment from process command line'}

## Stage 6 - load the model

In [15]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

In [16]:
load(algo_name+"_"+model_name)

{'hyperparameter': 0,
 'desc': 'msticpy ioc enrichment from process command line'}

## Stage 7 - provide a summary of the model

In [17]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} ,"desc": "msticpy ioc enrichment from process command line"}
    return returns

After implementing your fit, apply, save and load you can train your model:<br>

index=botsv2 "powershell" "-enc" source="WinEventLog*Microsoft-Windows-Sysmon*Operational*" <br>
| where LIKE(ParentCommandLine,"%powershell%-enc%") | stats count by ParentCommandLine <br>
| fit MLTKContainer algo=msticpy_powershell_ioc ParentCommandLine into app:process_b64_iocs_enrich<br>

Or apply your model:<br>

index=botsv2 "powershell" "-enc" source="WinEventLog*Microsoft-Windows-Sysmon*Operational*"<br>
| where LIKE(ParentCommandLine,"%powershell%-enc%") | stats count by ParentCommandLine<br>
| apply process_b64_iocs_enrich<br>

In [ ]:
summary()

{'version': {'numpy': '1.22.1', 'pandas': '1.5.3'},
 'desc': 'msticpy ioc enrichment from process command line'}

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [19]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [ ]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code